In [ ]:
# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Navigate to the dataset directory
cd /content/drive/MyDrive/RESEARCH/DATASET/MedicalExpert-I

In [ ]:
# Import necessary libraries and check TensorFlow version
import tensorflow
print(tensorflow.__version__)

In [ ]:
# Load dataset and prepare labels
import cv2,os
data_path='/content/drive/MyDrive/RESEARCH/DATASET/MedicalExpert-I'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]
label_dict=dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

In [ ]:
# Preprocess images: resize, convert to grayscale, and normalize
img_size=256
data=[]
label=[]

for category in categories:
  folder_path=os.path.join(data_path,category)
  # Check if folder_path is actually a directory before proceeding
  if os.path.isdir(folder_path):
    img_names=os.listdir(folder_path)

    for img_name in img_names:
      img_path=os.path.join(folder_path,img_name)
      img=cv2.imread(img_path)
      try:
          gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
          resized=cv2.resize(gray,(img_size,img_size))
          data.append(resized)
          label.append(label_dict[category])
      except Exception as e:
          print('Exception:',e)

In [ ]:
# Normalize image data and reshape for CNN input
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
label=np.array(label)
from tensorflow.keras.utils import to_categorical
new_label=to_categorical(label)

In [ ]:
# Convert labels to categorical format
new_label.shape

CNN MODEL


In [ ]:
# Print the shape of data
data.shape

In [ ]:
# Print the input shape for CNN model
data.shape[1:]

In [ ]:
# Build a Convolutional Neural Network (CNN) model
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()
model.add(Conv2D(128,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dense(5,activation='softmax'))
# Compile the CNN model

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Print the summary of the model
model.summary()

SPLITTING DATA INTO TRAINING AND TESTING


In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,new_label,test_size=0.1)

In [ ]:
# Visualize some test data samples
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(20):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(np.squeeze(x_test[i]))
  plt.xlabel(categories[np.argmax(y_test[i])])
plt.show()

In [ ]:
# Train the model and validate it
history=model.fit(x_train,y_train,epochs=100,validation_split=0.2)

In [ ]:
# Save the trained model
model.save('medical_expert.h5')

In [ ]:
# Plot training and validation loss and accuracy
from matplotlib import pyplot as plt

In [ ]:
N=100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N),history.history["loss"],label="train_loss")
plt.plot(np.arange(0,N),history.history["val_loss"],label="val_loss")
plt.plot(np.arange(0,N),history.history["accuracy"],label="train_acc")
plt.plot(np.arange(0,N),history.history["val_accuracy"],label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("CNN_Model")

In [ ]:
# Evaluate the model on the test data
val_loss,val_accuracy=model.evaluate(x_test,y_test,verbose=0)
print("test loss:",val_loss,'%')
print("test accuracy:",val_accuracy,'%')


In [ ]:
# Test the model on a single sample
X=32

img_size=256

img_single=x_test[X]
img_single=cv2.resize(img_single,(img_size,img_size))
img_single=(np.expand_dims(img_single,0))
img_single=img_single.reshape(img_single.shape[0],256,256,1)

predictions_single=model.predict(img_single)
print('A.I predits:',categories[np.argmax(predictions_single)])
print("correct predictions for label",categories[np.argmax(y_test[X])])
plt.imshow(np.squeeze(img_single))
plt.grid(False)
plt.show()

In [ ]:
# Generate a confusion matrix to evaluate the model
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

test_labels=np.argmax(y_test,axis=1)
predictions=model.predict(x_test)
predictions=np.argmax(predictions,axis=1)

cm=confusion_matrix(test_labels,predictions)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8),hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(5),['Normal','Doutful','Mild','Moderate','Severe'],fontsize=16)
plt.yticks(range(5),['Normal','Doutful','Mild','Moderate','Severe'],fontsize=16)
plt.show()